In [1]:
# 安装所需库（如未安装）：
# pip install ucimlrepo scikit-learn cvxpy pandas

from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd
import cvxpy as cp

# 1. 导入 bank marketing 数据集
bank = fetch_ucirepo(id=222)
X_raw = bank.data.features
y = bank.data.targets  # 暂时不使用

# 2. 选取数值型特征 + 抽样 4000 条数据
selected_features = ['age', 'balance', 'duration']
X_raw = X_raw.sample(n=4000, random_state=42).reset_index(drop=True)
X = X_raw[selected_features].copy()

# 3. 标准化特征
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 4. 提取敏感属性（如：marital）
sensitive_attr = X_raw['marital'].values
groups = np.unique(sensitive_attr)
group_indices = {g: np.where(sensitive_attr == g)[0] for g in groups}

# 5. 初始 KMeans 聚类
k = 4
kmeans = KMeans(n_clusters=k, random_state=0).fit(X_scaled)
initial_centers = kmeans.cluster_centers_

# 6. 构建距离矩阵（每个点到每个中心的距离）
n = X_scaled.shape[0]
dist_matrix = np.linalg.norm(X_scaled[:, np.newaxis, :] - initial_centers, axis=2)

# 7. 定义公平性参数（基于全局比例）
alpha = 1.2  # 最多不能超过整体比例的 120%
beta = 0.8   # 最少不能低于整体比例的 80%
ri = {g: len(idx)/n for g, idx in group_indices.items()}

# 8. 定义变量和优化目标
assign = cp.Variable((n, k), boolean=True)
objective = cp.Minimize(cp.sum(cp.multiply(assign, dist_matrix)))
constraints = [cp.sum(assign[i, :]) == 1 for i in range(n)]

# 加入公平性约束
for j in range(k):
    cluster_size = cp.sum(assign[:, j])
    for g in groups:
        idx = group_indices[g]
        group_assign = cp.sum(assign[idx, j])
        constraints.append(group_assign >= beta * ri[g] * cluster_size)
        constraints.append(group_assign <= alpha * ri[g] * cluster_size)

# 9. 求解公平再分配问题（使用 ECOS_BB）
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.ECOS_BB)

# 10. 提取最终聚类结果
if assign.value is not None:
    final_labels = np.argmax(assign.value, axis=1)
else:
    raise ValueError("优化失败，请检查公平性参数或换用其他求解器。")

# 11. 输出每个聚类中各群体分布（敏感属性为 marital）
print("\n✅ 聚类结果中各群体比例（敏感属性：marital）:")
for j in range(k):
    print(f"\nCluster {j}:")
    cluster_idx = np.where(final_labels == j)[0]
    for g in groups:
        g_count = np.sum(sensitive_attr[cluster_idx] == g)
        print(f"  {g}: {g_count / len(cluster_idx):.2f}")


/opt/anaconda3/lib/python3.12/site-packages/cvxpy/problems/problem.py:1504: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


ValueError: 优化失败，请检查公平性参数或换用其他求解器。

总结
推荐评判标准：
主要：正类召回率（优先捕捉潜在订阅客户）。
次要：正类 F1 分数（平衡精确率和召回率）。
辅助：AUC-ROC 和 AUPRC（评估整体性能和正类表现）。
理由：银行营销的目标是识别更多潜在订阅者，召回率直接衡量这一能力。F1 分数确保误报可控，AUC-ROC 和 AUPRC 提供全面性能评估。
后续步骤：运行上述代码，比较召回率和 F1 分数的提升。如果召回率仍不足，可进一步调整阈值或尝试 XGBoost。